In [1]:
import torch

In [2]:
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
import numpy as np
import pandas as pd
import torch.nn as nn
import os
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import utils
import importlib

In [4]:
pd.read_csv('hka-aqm-am/hka-aqm-am001_2022_09_05.dat', skiprows=1, sep=';', engine='python')

,date_time,device_id,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,rssi,channel_rssi,snr,gateway,channel_index,spreading_factor,bandwidth,f_cnt
0,2022-09-05 12:57:18,hka-aqm-am001,25.05,51.89,991,611,106,16,0,0,-127,-127,-18.2,drag-lps8-02,1,10,125000,2
1,2022-09-05 12:58:00,hka-aqm-am001,25.05,51.89,995,602,30,7,2,1,-128,-128,-15.5,drag-lps8-02,4,10,125000,3
2,2022-09-05 13:20:12,hka-aqm-am001,25.05,52.08,1000,544,109,20,2,0,-133,-133,-13.8,drag-lps8-01,2,10,125000,5
3,2022-09-05 13:35:18,hka-aqm-am001,25.17,51.79,728,450,109,17,2,0,-124,-124,-6.8,drag-lps8-02,5,10,125000,6
4,2022-09-05 13:50:25,hka-aqm-am001,25.30,51.98,670,465,95,14,2,0,-132,-132,-15.2,drag-lps8-02,0,10,125000,7
5,2022-09-05 14:05:31,hka-aqm-am001,25.39,51.54,646,518,96,14,3,1,-136,-136,-14.5,drag-outd-01,7,10,125000,8
6,2022-09-05 14:35:45,hka-aqm-am001,25.58,51.63,642,579,116,19,0,0,-134,-134,-15.5,drag-outd-01,1,10,125000,10
7,2022-09-05 15:47:39,hka-aqm-am001,25.71,50.27,617,633,25,9,1,0,-138,-138,-16.5,drag-lps8-01,7,10,125000,15


In [5]:
# load all files into one single df
# df = pd.concat([pd.read_csv('hka-aqm-am/' + f, skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [6]:
importlib.reload(utils)
train_df, test_df, train_loader, test_loader, scaler, y_test = utils.get_data_for_transformer(df, y_feature='CO2', aggregation_level='quarter_hour', window_size=20)

training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f'{y_feature}_scaled'] = scaler.fit_transform(df_train[[y_feature]])
c:\Studium\Semester_6\IoT_Project\utils.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{y_feature}_scaled'] = scaler.transform(df_test[[y_feature]])


Training data shape: torch.Size([244176, 20, 1]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 1]) torch.Size([62745, 1])


In [7]:
# print number of training samples
print(f"Number of training samples: {len(train_df)}")

Number of training samples: 244176


In [8]:
trained_model = utils.train_transformer_model(device, train_loader, test_loader, scaler, epochs=20)

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
c:\Users\t-eh

      actual   predicted
0      467.0  480.819824
1      456.0  482.360046
2      456.0  472.501251
3      438.0  473.990631
4      435.0  453.605591
...      ...         ...
1495   433.0  438.825378
1496   433.0  436.932220
1497   433.0  439.799469
1498   436.0  442.897644
1499   438.0  446.212952

[1500 rows x 2 columns]
Epoch 1/20, Validation Loss: 0.0214
      actual   predicted
0      467.0  482.822113
1      456.0  482.633728
2      456.0  469.796478
3      438.0  470.617706
4      435.0  450.834076
...      ...         ...
1495   433.0  438.832672
1496   433.0  436.056427
1497   433.0  437.165863
1498   436.0  439.317963
1499   438.0  443.869598

[1500 rows x 2 columns]
Epoch 2/20, Validation Loss: 0.0200
      actual   predicted
0      467.0  483.745819
1      456.0  483.471313
2      456.0  470.924591
3      438.0  472.360687
4      435.0  453.208344
...      ...         ...
1495   433.0  441.018951
1496   433.0  438.538177
1497   433.0  439.379700
1498   436.0  441.694336
149

In [9]:
utils.save_model(trained_model)
utils.save_scaler(scaler)

In [10]:
importlib.reload(utils)
utils.evaluate_transformer_model(device, test_loader, trained_model, scaler, y_test)

       actual   predicted
0       467.0  496.066132
1       456.0  497.933986
2       456.0  484.892290
3       438.0  484.071528
4       435.0  463.431870
...       ...         ...
62740   520.0  554.625692
62741   529.0  547.845096
62742   529.0  553.168707
62743   525.0  551.131596
62744   525.0  547.104759

[62745 rows x 2 columns]
Score (RMSE): 25.9574


In [11]:
importlib.reload(utils)
utils.predict_data(trained_model, scaler, df)

cpy:  (598590, 19)
help:  (309157, 11)
NaN count:  115351
                 date_time       device_id    tmp    hum   CO2  VOC  vis  IR  \
0      2022-09-05 12:57:18   hka-aqm-am001  25.05  51.89   991  611  106  16   
1      2022-09-05 12:58:00   hka-aqm-am001  25.05  51.89   995  602   30   7   
2      2022-09-05 13:20:12   hka-aqm-am001  25.05  52.08  1000  544  109  20   
3      2022-09-05 13:35:18   hka-aqm-am001  25.17  51.79   728  450  109  17   
4      2022-09-05 13:50:25   hka-aqm-am001  25.30  51.98   670  465   95  14   
...                    ...             ...    ...    ...   ...  ...  ...  ..   
598585 2023-09-26 22:50:49   hka-aqm-am308  27.38  37.58   520  950    7   3   
598586 2023-09-26 23:05:56   hka-aqm-am308  27.36  37.58   529  948    4   2   
598587 2023-09-26 23:21:03   hka-aqm-am308  27.35  37.61   529  966    4   0   
598588 2023-09-26 23:36:09   hka-aqm-am308  27.34  37.65   525  944    4   0   
598589 2023-09-26 23:51:16   hka-aqm-am308  27.33  37.65   525

,date_time,device_id,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,...,snr,gateway,channel_index,spreading_factor,bandwidth,f_cnt,date_time_rounded,CO2_context,CO2_context_unscaled,CO2_pred
0,2022-09-05 12:57:18,hka-aqm-am001,25.05,51.89,991,611,106,16,0,0,...,-18.2,drag-lps8-02,1,10,125000,2,2022-09-05 12:45:00,NaN,NaN,NaN
1,2022-09-05 12:58:00,hka-aqm-am001,25.05,51.89,995,602,30,7,2,1,...,-15.5,drag-lps8-02,4,10,125000,3,2022-09-05 12:45:00,NaN,NaN,NaN
2,2022-09-05 13:20:12,hka-aqm-am001,25.05,52.08,1000,544,109,20,2,0,...,-13.8,drag-lps8-01,2,10,125000,5,2022-09-05 13:15:00,NaN,NaN,NaN
3,2022-09-05 13:35:18,hka-aqm-am001,25.17,51.79,728,450,109,17,2,0,...,-6.8,drag-lps8-02,5,10,125000,6,2022-09-05 13:30:00,NaN,NaN,NaN
4,2022-09-05 13:50:25,hka-aqm-am001,25.30,51.98,670,465,95,14,2,0,...,-15.2,drag-lps8-02,0,10,125000,7,2022-09-05 13:45:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598585,2023-09-26 22:50:49,hka-aqm-am308,27.38,37.58,520,950,7,3,2,0,...,12.2,drag-lps8-05,4,8,125000,12637,2023-09-26 22:45:00,"[0.13122275037653255, 0.15266064368583365, 0.1...","[507.0, 510.0, 514.0, 511.0, 513.0, 513.0, 513...",546.355164
598586,2023-09-26 23:05:56,hka-aqm-am308,27.36,37.58,529,948,4,2,1,0,...,10.0,drag-lps8-05,6,8,125000,12638,2023-09-26 23:00:00,"[0.15266064368583365, 0.18124450143156842, 0.1...","[510.0, 514.0, 511.0, 513.0, 513.0, 513.0, 518...",540.666809
598587,2023-09-26 23:21:03,hka-aqm-am308,27.35,37.61,529,966,4,0,6,27,...,11.8,drag-lps8-05,0,8,125000,12639,2023-09-26 23:15:00,"[0.18124450143156842, 0.15980660812226735, 0.1...","[514.0, 511.0, 513.0, 513.0, 513.0, 518.0, 516...",551.074890
598588,2023-09-26 23:36:09,hka-aqm-am308,27.34,37.65,525,944,4,0,0,0,...,12.0,drag-lps8-05,3,8,125000,12640,2023-09-26 23:30:00,"[0.15980660812226735, 0.17409853699513472, 0.1...","[511.0, 513.0, 513.0, 513.0, 518.0, 516.0, 517...",550.401428


In [12]:
df.describe()

,tmp,hum,CO2,VOC,vis,IR,WIFI,BLE,rssi,channel_rssi,channel_index,spreading_factor,bandwidth,f_cnt
count,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.000000,608036.0,608036.000000
mean,25.142613,36.724730,607.723873,925.523094,801.135497,227.006169,2.476212,5.361508,-95.298145,-95.298145,3.497678,7.568132,125000.0,7406.347787
std,4.123532,9.568186,1058.630803,1161.608153,3225.759145,1355.515628,1.743450,11.378924,21.379657,21.379657,2.292534,0.683923,0.0,7658.187112
min,6.920000,7.760000,264.000000,447.000000,0.000000,0.000000,0.000000,0.000000,-139.000000,-139.000000,0.000000,7.000000,125000.0,1.000000
25%,22.730000,29.650000,421.000000,611.000000,8.000000,1.000000,1.000000,0.000000,-113.000000,-113.000000,1.000000,7.000000,125000.0,1221.000000
50%,24.650000,36.120000,448.000000,675.000000,121.000000,28.000000,2.000000,1.000000,-99.000000,-99.000000,3.000000,7.000000,125000.0,4477.000000
75%,27.600000,43.030000,506.000000,858.000000,634.000000,159.000000,4.000000,5.000000,-79.000000,-79.000000,5.000000,8.000000,125000.0,11820.000000
max,583.720000,622.150000,24001.000000,21930.000000,65535.000000,46896.000000,13.000000,128.000000,-32.000000,-32.000000,7.000000,12.000000,125000.0,37271.000000


In [22]:
importlib.reload(utils)
_, _ = utils.create_transformer_model_for_feature(df, 'CO2', 'quarter_hour', device, 20, epochs=20)

training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f'{y_feature}_scaled'] = scaler.fit_transform(df_train[[y_feature]])
c:\Studium\Semester_6\IoT_Project\utils.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{y_feature}_scaled'] = scaler.transform(df_test[[y_feature]])


Training data shape: torch.Size([244176, 20, 1]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 1]) torch.Size([62745, 1])


c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


      actual   predicted
0      467.0  447.693756
1      456.0  442.688721
2      456.0  430.759033
3      438.0  430.287354
4      435.0  423.127441
...      ...         ...
1495   433.0  442.244263
1496   433.0  444.610687
1497   433.0  445.418152
1498   436.0  444.024689
1499   438.0  443.407501

[1500 rows x 2 columns]
Epoch 1/20, Validation Loss: 0.0285
      actual   predicted
0      467.0  457.338043
1      456.0  452.740143
2      456.0  437.351227
3      438.0  437.728882
4      435.0  425.677338
...      ...         ...
1495   433.0  435.190826
1496   433.0  436.827179
1497   433.0  437.795410
1498   436.0  437.143188
1499   438.0  438.042175

[1500 rows x 2 columns]
Epoch 2/20, Validation Loss: 0.0169
      actual   predicted
0      467.0  470.389862
1      456.0  466.821167
2      456.0  452.604370
3      438.0  453.082581
4      435.0  438.673431
...      ...         ...
1495   433.0  442.866272
1496   433.0  443.023193
1497   433.0  443.075897
1498   436.0  441.476196
149

In [23]:
importlib.reload(utils)
_, _ = utils.create_transformer_model_for_feature(df, 'tmp', 'quarter_hour', device, 20, epochs=20)

training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f'{y_feature}_scaled'] = scaler.fit_transform(df_train[[y_feature]])
c:\Studium\Semester_6\IoT_Project\utils.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{y_feature}_scaled'] = scaler.transform(df_test[[y_feature]])


Training data shape: torch.Size([244176, 20, 1]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 1]) torch.Size([62745, 1])


c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


         actual  predicted
0     26.090000  25.701118
1     26.010000  25.672993
2     25.980000  25.578171
3     25.950001  25.594568
4     25.920000  25.650827
...         ...        ...
1495  25.129999  24.878340
1496  25.129999  24.973904
1497  25.020000  24.925747
1498  25.160000  24.716673
1499  25.190001  24.785969

[1500 rows x 2 columns]
Epoch 1/20, Validation Loss: 0.0356
         actual  predicted
0     26.090000  25.797909
1     26.010000  25.776945
2     25.980000  25.667952
3     25.950001  25.675945
4     25.920000  25.712330
...         ...        ...
1495  25.129999  24.832352
1496  25.129999  24.920134
1497  25.020000  24.900494
1498  25.160000  24.729099
1499  25.190001  24.802853

[1500 rows x 2 columns]
Epoch 2/20, Validation Loss: 0.0701
         actual  predicted
0     26.090000  25.674683
1     26.010000  25.673864
2     25.980000  25.575512
3     25.950001  25.572414
4     25.920000  25.577925
...         ...        ...
1495  25.129999  24.717865
1496  25.12999

In [24]:
importlib.reload(utils)
_, _ = utils.create_transformer_model_for_feature(df, 'hum', 'quarter_hour', device, 20, epochs=20)

training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f'{y_feature}_scaled'] = scaler.fit_transform(df_train[[y_feature]])
c:\Studium\Semester_6\IoT_Project\utils.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{y_feature}_scaled'] = scaler.transform(df_test[[y_feature]])


Training data shape: torch.Size([244176, 20, 1]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 1]) torch.Size([62745, 1])


c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


         actual  predicted
0     45.490002  41.162346
1     45.009998  42.166889
2     44.880001  41.538807
3     44.840000  41.276741
4     45.360001  41.102859
...         ...        ...
1495  43.939999  40.267654
1496  44.000000  40.422832
1497  45.070000  40.503162
1498  44.779999  41.593277
1499  44.779999  41.108410

[1500 rows x 2 columns]
Epoch 1/20, Validation Loss: 0.0992
         actual  predicted
0     45.490002  40.774258
1     45.009998  41.634472
2     44.880001  41.002193
3     44.840000  40.703251
4     45.360001  40.541958
...         ...        ...
1495  43.939999  40.263062
1496  44.000000  40.311016
1497  45.070000  40.317562
1498  44.779999  41.295650
1499  44.779999  40.926128

[1500 rows x 2 columns]
Epoch 2/20, Validation Loss: 0.1309
         actual  predicted
0     45.490002  40.050343
1     45.009998  40.860771
2     44.880001  40.212975
3     44.840000  39.932571
4     45.360001  39.806309
...         ...        ...
1495  43.939999  39.313141
1496  44.00000

In [25]:
importlib.reload(utils)
_, _ = utils.create_transformer_model_for_feature(df, 'VOC', 'quarter_hour', device, 20, epochs=20)

training data cutoff:  2023-07-15 02:45:00


c:\Studium\Semester_6\IoT_Project\utils.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f'{y_feature}_scaled'] = scaler.fit_transform(df_train[[y_feature]])
c:\Studium\Semester_6\IoT_Project\utils.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{y_feature}_scaled'] = scaler.transform(df_test[[y_feature]])


Training data shape: torch.Size([244176, 20, 1]) torch.Size([244176, 1])
Testing data shape: torch.Size([62745, 20, 1]) torch.Size([62745, 1])


c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


      actual    predicted
0     1133.0  1146.250854
1      955.0  1204.773438
2      990.0   937.145813
3      811.0   989.455750
4      739.0   747.278137
...      ...          ...
1495  1596.0  1643.941528
1496  1651.0  1734.276978
1497  1343.0  1792.824463
1498  1506.0  1408.577637
1499  1575.0  1574.317261

[1500 rows x 2 columns]
Epoch 1/20, Validation Loss: 0.0814
      actual    predicted
0     1133.0  1109.152832
1      955.0  1152.550171
2      990.0   928.406616
3      811.0   970.460205
4      739.0   758.133667
...      ...          ...
1495  1596.0  1640.792603
1496  1651.0  1713.834839
1497  1343.0  1761.409424
1498  1506.0  1412.154175
1499  1575.0  1590.524780

[1500 rows x 2 columns]
Epoch 2/20, Validation Loss: 0.0774
      actual    predicted
0     1133.0  1072.286621
1      955.0  1110.765991
2      990.0   913.545532
3      811.0   952.416626
4      739.0   766.163513
...      ...          ...
1495  1596.0  1545.783813
1496  1651.0  1621.594360
1497  1343.0  1666.5